In [12]:
#使用外部request库
import requests
import webbrowser


#1、requests.get(url, params=None, headers=None, cookies=None, auth=None, timeout=None)
#参数：
# url - 新的Request对象的URL。
# params - （可选）使用请求发送的GET参数字典。
# 头文件 - （可选）使用请求发送的HTTP头文件字典。
# Cookie - （可选）使用请求发送的CookieJar对象。
# auth - （可选）AuthObject启用基本HTTP身份验证。
# 超时 - （可选）描述请求超时的浮点数。

#返回一个携带url的对象

param = {"wd": "莫烦Python"}  # 搜索的信息
r = requests.get('http://www.baidu.com/s', params=param)
print(r.url)
#print(r.text)
#webbrowser.open(r.url)

http://www.baidu.com/s?wd=%E8%8E%AB%E7%83%A6Python


In [10]:
#post方法
data = {'UserName': '201631003119', 'UserPass': '13379070wyx','CheckCode':' '}
r = requests.post('http://bsdt.swun.edu.cn/yhmm.aspx', data=data)
print(r.url)
print(r.status_code)

http://bsdt.swun.edu.cn/yhmm.aspx
200
<PreparedRequest [POST]>


In [13]:
file = {'uploadFile': open('./image.png', 'rb')}
r = requests.post('http://pythonscraping.com/files/processing2.php', files=file)
print(r.text)

FileNotFoundError: [Errno 2] No such file or directory: './image.png'

In [4]:
#模拟登陆，使用cookies
import requests
import webbrowser
payload = {'username': 'Morvan', 'password': 'password'}
r = requests.post('http://pythonscraping.com/pages/cookies/welcome.php', data=payload)
print(r.cookies.get_dict())

# {'username': 'Morvan', 'loggedin': '1'}


r = requests.get('http://pythonscraping.com/pages/cookies/profile.php', cookies=r.cookies)
webbrowser.open(r.url)

{'loggedin': '1', 'username': 'Morvan'}


True

In [9]:
#使用session进行登录
session = requests.Session()
payload = {'username': 'Morvan', 'password': 'password'}
r = session.post('http://pythonscraping.com/pages/cookies/welcome.php', data=payload)
print(r.cookies.get_dict())
r = session.get("http://pythonscraping.com/pages/cookies/profile.php")
#print(r.text)

{'loggedin': '1', 'username': 'Morvan'}


In [11]:
#下载文件
import os 

os.makedirs('./download_img',exist_ok=True)
IMAGE_URL = "https://morvanzhou.github.io/static/img/description/learning_step_flowchart.png"

#方法1 使用urlretrieve
from urllib.request import urlretrieve
urlretrieve(IMAGE_URL,'./download_img/img1.png')

#方法2 使用request，这种方法更好一些，类似于断点续传.可以下载大文件
r = requests.get(IMAGE_URL)
with open('./download_img/img2.png','wb') as f:
    f.write(r.content)


In [12]:
#下载大文件的时候可以使用iter_content()来指定chunk的大小
r = requests.get(IMAGE_URL, stream=True)    # stream loading

with open('./download_img/image3.png', 'wb') as f:
    for chunk in r.iter_content(chunk_size=32):
        f.write(chunk)

In [23]:
#例子
import os
import re
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup

#目标网址
Target_url = "http://www.ngchina.com.cn/animals/"
img_num = 0
#获取内容
html = urlopen(Target_url).read().decode('utf-8')

#解析网页
soup = BeautifulSoup(html,features='lxml')

#正则表达式筛选图片url
sub_urls = soup.find_all('img',{"src":re.compile("http://image.ngchina.com.cn/2019/.*jpg$")})

#开始下载
for url in sub_urls: 
    r = requests.get(url['src'])
    with open('./download_img/%d.png' % img_num, 'wb') as f:
        for chunk in r.iter_content(chunk_size=128):
            f.write(chunk)
    print("Download successfully: %d.jpg" % img_num)
    img_num += 1

Download successfully: 0.jpg
Download successfully: 1.jpg
Download successfully: 2.jpg
Download successfully: 3.jpg
Download successfully: 4.jpg
Download successfully: 5.jpg
Download successfully: 6.jpg
Download successfully: 7.jpg
Download successfully: 8.jpg
Download successfully: 9.jpg
Download successfully: 10.jpg
Download successfully: 11.jpg
Download successfully: 12.jpg
Download successfully: 13.jpg
Download successfully: 14.jpg
Download successfully: 15.jpg
Download successfully: 16.jpg
Download successfully: 17.jpg
Download successfully: 18.jpg
Download successfully: 19.jpg
Download successfully: 20.jpg
Download successfully: 21.jpg
Download successfully: 22.jpg
Download successfully: 23.jpg
Download successfully: 24.jpg
Download successfully: 25.jpg
Download successfully: 26.jpg
Download successfully: 27.jpg
Download successfully: 28.jpg
Download successfully: 29.jpg
Download successfully: 30.jpg
Download successfully: 31.jpg
Download successfully: 32.jpg
Download successfull

In [24]:
###重要，多进程进行分布式爬虫，更加有效率
import multiprocessing as mp
import requests
import time
import re
from bs4 import BeautifulSoup
from urllib.request import urlopen, urljoin

base_url = 'https://morvanzhou.github.io/'

if base_url != "https://morvanzhou.github.io/":
    restricted_crawl = True
else:
    restricted_crawl = False

#函数定义
#1.爬取网页 crawl()
def crawl(url):
    response = urlopen(url).read().decode('utf-8')
    return response

#2.网页解析
def parse(html):
    soup = BeautifulSoup(html, 'lxml')
    urls = soup.find_all('a', {"href": re.compile('^/.+?/$')})
    title = soup.find('h1').get_text().strip()
    page_urls = set([urljoin(base_url, url['href']) for url in urls])   # 去重
    url = soup.find('meta', {'property': "og:url"})['content']
    return title, page_urls, url
    
unseen = set([base_url,])
seen = set()

count, t1 = 1, time.time()

if __name__ == "__main__":
    while len(unseen) != 0:                 # still get some url to visit
        if restricted_crawl and len(seen) > 20:
                break

        print('\nDistributed Crawling...')
        htmls = [crawl(url) for url in unseen]

        print('\nDistributed Parsing...')
        results = [parse(html) for html in htmls]

        print('\nAnalysing...')
        seen.update(unseen)         # seen the crawled
        unseen.clear()              # nothing unseen

        for title, page_urls, url in results:
            print(count, title, url)
            count += 1
            unseen.update(page_urls - seen)     # get new url to crawl
    print('Total time: %.1f s' % (time.time()-t1, ))    # 53 s  
    


Distributed Crawling...

Distributed Parsing...

Analysing...
1 教程 https://morvanzhou.github.io/

Distributed Crawling...

Distributed Parsing...

Analysing...
2 为了更优秀 https://morvanzhou.github.io/support/
3 迁移学习 Transfer Learning https://morvanzhou.github.io/tutorials/machine-learning/tensorflow/5-16-transfer-learning/
4 Sklearn 通用机器学习 教程系列 https://morvanzhou.github.io/tutorials/machine-learning/sklearn/
5 关于莫烦 https://morvanzhou.github.io/about/
6 Tensorflow 教程系列 https://morvanzhou.github.io/tutorials/machine-learning/tensorflow/
7 进化算法 Evolutionary Strategies 教程系列 https://morvanzhou.github.io/tutorials/machine-learning/evolutionary-algorithm/
8 说吧~ https://morvanzhou.github.io/discuss/
9 迁移学习 Transfer Learning https://morvanzhou.github.io/tutorials/machine-learning/ML-intro/2-9-transfer-learning/
10 Python基础 教程系列 https://morvanzhou.github.io/tutorials/python-basic/
11 基础教程系列 https://morvanzhou.github.io/tutorials/python-basic/basic/
12 网页爬虫教程系列 https://morvanzhou.github.io/tutorial

KeyboardInterrupt: 

In [ ]:
unseen = set([base_url,])
seen = set()

pool = mp.Pool(4)                       
count, t1 = 1, time.time()
while len(unseen) != 0:                 # still get some url to visit
    if restricted_crawl and len(seen) > 20:
            break
    print('\nDistributed Crawling...')
    crawl_jobs = [pool.apply_async(crawl, args=(url,)) for url in unseen]
    htmls = [j.get() for j in crawl_jobs]                                       # request connection

    print('\nDistributed Parsing...')
    parse_jobs = [pool.apply_async(parse, args=(html,)) for html in htmls]
    results = [j.get() for j in parse_jobs]                                     # parse html

    print('\nAnalysing...')
    seen.update(unseen)         # seen the crawled
    unseen.clear()              # nothing unseen

    for title, page_urls, url in results:
        print(count, title, url)
        count += 1
        unseen.update(page_urls - seen)     # get new url to crawl
print('Total time: %.1f s' % (time.time()-t1, ))    # 16 s !!!


Distributed Crawling...
